In [1]:
import numpy as np
# constant size
size = 1000
# Set p(H) apriori for simulation as a biased coin.
p_h = 0.7

# Model experiment as a single biased coin flipped 1000 times.
data = np.random.binomial(1, p_h, size=size)

In [2]:
n_states = 2
n_obs = 2

emission = np.array([[0.2,0.8],[0.3,0.7]])
transition = np.array([[0.5,0.5],[0.5,0.5]])


forward = np.random.rand(size,n_states)
scale_factors = np.zeros((size))
forward_hat = np.zeros((size,n_states))

backward = np.random.rand(size,n_states)
backward_hat = np.zeros((size,n_states))

init_prob = np.array([0.5,0.5])
init_obs = np.zeros(n_states)

p_old = -10000
tol = 0.001
max_iter = 500

In [6]:
for ite in range(max_iter):
    forward[0,:] = init_prob * emission[:,data[0]]
    scale_factors[0] = np.sum(forward[0,:])
    forward_hat[0,:] = forward[0,:]/scale_factors[0]
    for t in range(size-1):
        temp = np.matmul(forward_hat[t,:] ,transition) * emission[:,data[t]]
        scale_factors[t+1] = np.sum(temp)
        forward_hat[t+1,:] = temp/scale_factors[t+1]
        forward[t+1,:] = forward_hat[t+1,:]*np.prod(scale_factors[0:t+1])

    backward[-1,:] = 1
    backward_hat[-1,:] = backward[-1,:]
    for t in reversed(range(size-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[:,data[t+1]],np.transpose(transition))
        backward_hat[t,:] = temp/scale_factors[t+1]
        backward[t,:] = backward_hat[t,:]*np.prod(scale_factors[t+1:-1])


    a = np.zeros((size,n_states))
    b = np.zeros((size,n_states,n_states))
    for i in range(size):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
    for t in range(size-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]
    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])

    for i in range(n_states):
        for j in range(n_obs):
            emission[i,j] = np.sum(a[np.argwhere(data==j),i]) / np.sum(a[:,i])
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p}')
    # if p - p_old < tol:
    #     break
    p_old = p

p is:-604.0823015163592
p is:-738.8401007735116
p is:-604.0823018515571
p is:-738.8400998385664
p is:-604.0823017107292
p is:-738.8400988921001
p is:-604.0823015696288
p is:-738.8400979440974
p is:-604.0823014282994
p is:-738.8400969945571
p is:-604.0823012867411
p is:-738.840096043477
p is:-604.0823011449529
p is:-738.8400950908546
p is:-604.0823010029351
p is:-738.840094136687
p is:-604.0823008606868
p is:-738.8400931809717
p is:-604.0823007182079
p is:-738.8400922237065
p is:-604.0823005754978
p is:-738.8400912648888
p is:-604.0823004325566
p is:-738.8400903045164
p is:-604.0823002893835
p is:-738.8400893425866
p is:-604.0823001459785
p is:-738.8400883790966
p is:-604.0823000023408
p is:-738.8400874140442
p is:-604.0822998584704
p is:-738.8400864474269
p is:-604.0822997143664
p is:-738.8400854792421
p is:-604.0822995700291
p is:-738.8400845094875
p is:-604.0822994254577
p is:-738.8400835381599
p is:-604.0822992806518
p is:-738.8400825652577
p is:-604.0822991356113
p is:-738.84008159

In [9]:
emission

array([[0.24997555, 0.75002445],
       [0.25002581, 0.74997419]])

In [5]:
# transition_est = np.random.rand(size,n_states,n_states)
# for t in range(size-1):
#     for i in range(n_states):
#         for j in range(n_states):
#             transition_est[t,i,j] = forward[t,i]*transition[i,j]*emission[j,data[t+1]]*backward[t+1,j] / (np.sum(forward[t,:]*backward[t,:]))
